In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt
%matplotlib inline
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from scipy.stats import norm, sem
from scipy.interpolate import UnivariateSpline
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.stats import pearsonr
from numpy.random import default_rng

import seaborn as sns
from pycebox.ice import ice, ice_plot
from matplotlib.cm import seismic

from sympy import *

ModuleNotFoundError: No module named 'pycebox'

In [ ]:
x1,x2,x3,y = [],[],[],[]
N = 1000

for i in range(0,N):
    X1 = np.random.uniform(-1, 1, size = 1)
    X2 = np.random.uniform(-1, 1, size = 1)
    X3 = np.random.uniform(-1, 1, size = 1)
    Y = 0.2*X1 - 5*X2 + 10*X3

    x1.append(X1)
    x2.append(X2)
    x3.append(X3)    
    y.append(Y)

In [ ]:
data = np.concatenate([x1,x2,x3,y], axis = 1)
df = pd.DataFrame(data, columns=['X1','X2','X3','Y'])

In [ ]:
df

In [ ]:
X = df[['X1','X2','X3']]
X1, X2, X3 = df.X1, df.X2, df.X3
y = df.Y

std = {}

In [ ]:
minus_X, plus_X, minus_y, plus_y = train_test_split(X, y, test_size=.5, random_state=42)
minus_df = minus_X.join(minus_y)
plus_df = plus_X.join(plus_y)

In [ ]:
def change_features(X_og,X,i,alpha,group):
    X_list = []
    y_list = []

    for j in range(len(X)):
        new_values = {}
        std[i] = np.std(X[i])
        og_value = X[i].iloc[j]
        if group == 'plus':
            new_value = og_value + alpha*std[i]
        if group == 'minus':
            new_value = og_value - alpha*std[i]
        new_values[i] = new_value
        X_list.append(new_values)
    X_new = pd.DataFrame(X_list)
    X_new = X_new.join(X.reset_index().drop(columns=[i,'index']))
    for x in range(len(X_new)):
        y_list.append(0.2*X_new.X1[x] - 5*X_new.X2[x] + 10*X_new.X3[x]) 
    y_new = pd.DataFrame(y_list, columns=['Y'])
    df_new = X_new.join(y_new).append(X_og).reset_index().drop(columns=['index'])
    return(df_new)

In [ ]:
plus_df_X1 = change_features(minus_df, plus_X, 'X1', 0.05, 'plus')
minus_df_X1 = change_features(plus_df, minus_X, 'X1', 0.05, 'minus')
minus_df_X1

In [ ]:
def cate(y, y_new):
    mean0 = np.mean(y)
    mean1 = np.mean(y_new)
    tao = mean1 - mean0
    return tao

In [ ]:
minus_cate = {}
for x in minus_X:
    minus_df_new = change_features(minus_df, minus_X, x, 0.05, 'minus')
    c = cate(minus_y, minus_df_new.Y)
    minus_cate[x] = c
pd.DataFrame([minus_cate])

In [ ]:
plus_cate = {}
for x in plus_X:
    plus_df_new = change_features(minus_df, plus_X, x, 0.05, 'plus')
    c = cate(plus_y, plus_df_new.Y)
    plus_cate[x] = c
pd.DataFrame([plus_cate])

### Cancer test data

In [ ]:
X = pd.read_csv('cervical_cancer_risks/X.csv')
y = pd.read_csv('cervical_cancer_risks/y.csv')
df = X.join(y)

In [ ]:
df

In [ ]:
bin_col = []
for i in X:
    if  len(X[i].unique())<4:
        bin_col.append(i)

In [ ]:
bin_col

In [ ]:
for i in range(len(bin_col)):
    print(X[bin_col[i]].value_counts())

In [ ]:
def change_binary(df, X_feat, alpha):
    # alpha: percentage of values to change
    # Only keep binary variables
    binary = [0,1]
    df = df[df[X_feat].isin(binary)].reset_index()
    
    df_X = df.iloc[:,:-1]
    # Choose random indices to change values of
    rand = []
    m = int(np.floor(len(df_X)*alpha))
    for i in range(m):
        rand.append(random.randint(0,max(df_X.index)))
    for d in rand:
        df_X[X_feat].loc[d] = np.abs(df_X[X_feat].loc[d] - 1)
    
    rf = RandomForestClassifier()
    rf.fit(df.iloc[:,:-1], df.Biopsy)
    y_new = pd.DataFrame(rf.predict(df_X), columns=['pred_y'])
    df_new = df_X.join(y_new).reset_index().drop(columns=['index'])
    return(df_new)

In [ ]:
def change_continuous(df, X_feat, alpha, sig):
    # sig: value to change sigma by
    df_X = df.iloc[:,:-1]
    # Choose random indices to change values of
    std = np.std(df_X[X_feat])
    rand = []
    m = int(np.floor(len(df_X)*alpha))
    for i in range(m):
        rand.append(random.randint(0,max(df_X.index)))
    for d in rand:
        df_X[X_feat].loc[d] = df_X[X_feat].loc[d] + sig * std
    
    rf = RandomForestClassifier()
    rf.fit(df.iloc[:,:-1], df.Biopsy)
    y_new = pd.DataFrame(rf.predict(df_X), columns=['pred_y'])
    df_new = df_X.join(y_new).reset_index().drop(columns=['index'])
    return(df_new)

In [ ]:
def cate(y, y_new):
    mean0 = np.mean(y)
    mean1 = np.mean(y_new)
    tao = mean1 - mean0
    return tao

In [ ]:
X = df.iloc[:,:-1]
cate_values = {}
for X_feat in X:
    if X_feat in bin_col:
        df_new = change_binary(df, X_feat, alpha=0.5)
    else:
        df_new = change_continuous(df, X_feat, alpha=0.5, sig=1)
    c = cate(df.Biopsy, df_new.pred_y)
    cate_values[X_feat] = c
pd.DataFrame([cate_values])